In [54]:
import requests
import json
import pandas as pd

# Endpoint and API details
host = "https://rpc.nethermind.io/mainnet-juno"
JUNO_RPC_URL = host
stark_address = "0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d"

headers = {
    'x-apikey': '',
    'Content-Type': 'application/json'
}

# Define the payload as a dictionary (not a JSON string)
payload = {
    "jsonrpc": "2.0",
    "method": "starknet_getEvents",
    "params": {
        "filter": {
            "from_block": {"block_number": 1136080},
            "to_block": "latest",
            "address": stark_address,
            "keys": [["0x99cd8bde557814842a3121e8ddfd433a539b8c9f14bf31ebf108d12e6196e9"]],
            "chunk_size": 2
        }
    },
    "id": 1
}

all_events = []

# First request
response = requests.post(JUNO_RPC_URL, headers=headers, json=payload)
data = response.json()
result = data.get("result", {})
all_events.extend(result.get("events", []))
continuation_token = result.get("continuation_token")

while continuation_token:
    # Update the payload dictionary with the new continuation token and increment id
    payload["params"]["filter"]["continuation_token"] = continuation_token
    payload["id"] += 1
    response = requests.post(JUNO_RPC_URL, headers=headers, json=payload)
    data = response.json()
    result = data.get("result", {})
    all_events.extend(result.get("events", []))
    continuation_token = result.get("continuation_token")

# Process the events
formatted_events = []
for event in all_events:
    d = event.get("data", [])
    if len(d) >= 3:
        formatted_events.append({
            "from": d[0],
            "to": d[1],
            "value": float(int(d[2], 16) / (10 ** 18)),
            "block_number": event.get("block_number"),
            "block_hash": event.get("block_hash"),
            "transaction_hash": event.get("transaction_hash")
        })

# Create a DataFrame (optional, for further analysis)
df = pd.DataFrame(formatted_events)
df2 = df[df['from'] == stark_address]
# Print the events in JSON format
print(json.dumps({"events": formatted_events}, indent=2))

{
  "events": [
    {
      "from": "0x57ae2091dda992629b084fb29d5bd451e7aa50e78279ee30d50819b2e1d329f",
      "to": "0x1176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8",
      "value": 0.001327584757147766,
      "block_number": 1136070,
      "block_hash": "0x58ee6344643d1192c7b35291e998ad063de393b2aa01b93efafd53e2c8119b7",
      "transaction_hash": "0x7eab0acb8b67be3785a492ca7e1d1d33eddf3c50720d344647d978f1d958619"
    },
    {
      "from": "0x40ce8a775194a5c89d718ebc55cde1cc81f41944d571e6b5fab341e00c3164d",
      "to": "0x5d2a429a2fb847d939b1178e26007f7b14437b2c950e5317e165a160406b347",
      "value": 9.437550791685936,
      "block_number": 1136070,
      "block_hash": "0x58ee6344643d1192c7b35291e998ad063de393b2aa01b93efafd53e2c8119b7",
      "transaction_hash": "0x57dc54e02233c55370e9721f9ec65c3c48663e5af9c9267cf5aa26b52ac3653"
    },
    {
      "from": "0x5d2a429a2fb847d939b1178e26007f7b14437b2c950e5317e165a160406b347",
      "to": "0x1176a1bd84444c89232ec277546

In [41]:
df2 = df[df['from'] == "0x34d2a751fb800f42106cb1b05e90ac50a912c0737f03125c6ae0f5bc090a98e"]
df.head()


,from,to,value,block_number,block_hash,transaction_hash
0,0x57ae2091dda992629b084fb29d5bd451e7aa50e78279ee30d50819b2e1d329f,0x1176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8,0.001328,1136070,0x58ee6344643d1192c7b35291e998ad063de393b2aa01b93efafd53e2c8119b7,0x7eab0acb8b67be3785a492ca7e1d1d33eddf3c50720d344647d978f1d958619
1,0x40ce8a775194a5c89d718ebc55cde1cc81f41944d571e6b5fab341e00c3164d,0x5d2a429a2fb847d939b1178e26007f7b14437b2c950e5317e165a160406b347,9.437551,1136070,0x58ee6344643d1192c7b35291e998ad063de393b2aa01b93efafd53e2c8119b7,0x57dc54e02233c55370e9721f9ec65c3c48663e5af9c9267cf5aa26b52ac3653
2,0x5d2a429a2fb847d939b1178e26007f7b14437b2c950e5317e165a160406b347,0x1176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8,0.001135,1136070,0x58ee6344643d1192c7b35291e998ad063de393b2aa01b93efafd53e2c8119b7,0x57dc54e02233c55370e9721f9ec65c3c48663e5af9c9267cf5aa26b52ac3653
3,0x105efaf4071aca0e0b873b1d82800a8cf511a2834e72fbf55dd597424d8441f,0x7e78e986058ddbcdcd2c422b40351eccf780c517ec70185e1cef3426f353512,0.591225,1136070,0x58ee6344643d1192c7b35291e998ad063de393b2aa01b93efafd53e2c8119b7,0x6a7bd93acd5c75e9406b7353441f040ffd46c0761187c566de92a33ee5b0355
4,0x4bc7405c31eb772706d972d88349d297a493f41004e9e9181b6d4039ee39fa,0x7e78e986058ddbcdcd2c422b40351eccf780c517ec70185e1cef3426f353512,0.158424,1136070,0x58ee6344643d1192c7b35291e998ad063de393b2aa01b93efafd53e2c8119b7,0x6a7bd93acd5c75e9406b7353441f040ffd46c0761187c566de92a33ee5b0355
